In [ ]:
import logging

from rich.logging import RichHandler
from rich.traceback import install

from darts.utils.earthengine import init_ee
from darts.utils.logging import LoggingManager

LoggingManager.setup_logging()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)
install(
    show_locals=False, locals_hide_dunder=True, locals_hide_sunder=True
)  # Change to False if you encounter too large tracebacks
init_ee("ee-tobias-hoelzer", use_highvolume=False)  # Replace with your project

In [ ]:
import importlib
from inspect import getmembers, isclass, isfunction
from pathlib import Path

modules = [
    "darts.automated_pipeline",  # -> DARTS Automated Pipeline
    "darts",  # -> DARTS
    "darts.legacy_pipeline",  # -> DARTS Legacy Pipeline
    "darts.legacy_training",  # -> DARTS Legacy Training
    "darts_acquisition",
    "darts_ensemble",
    "darts_export",
    "darts_postprocessing",
    "darts_preprocessing",
    "darts_segmentation",
    "darts_segmentation.training",
    "darts_segmentation.metrics",
    # "darts_superresolution",
    # "darts_utils.cuda",
    # "darts_utils.rich",
    # "darts_utils.stopuhr",
    # TODO: dart.utils and darts_utils
]

for module in modules:
    module_docpath = Path("../docs/ref") / module.replace(".", "/")
    module_docpath.mkdir(parents=True, exist_ok=True)
    modulepy = importlib.import_module(module)

    toc = []
    for funcname, _ in getmembers(modulepy, isfunction):
        function_docfile = module_docpath / f"{funcname}.md"
        function_docfile.write_text(f"::: {module}.{funcname}\n")
        # tocpath = function_docfile.relative_to("../docs")
        tocpath = function_docfile.name
        toc.append(f"[{funcname}]({tocpath})\n")
    for classname, _ in getmembers(modulepy, isclass):
        class_docfile = module_docpath / f"{classname}.md"
        class_docfile.write_text(f"::: {module}.{classname}\n")
        # tocpath = class_docfile.relative_to("../docs")
        tocpath = class_docfile.name
        toc.append(f"- [{classname}]({(tocpath)})\n")

    module_docfile = module_docpath / "index.md"
    with open(module_docfile, "w") as f:
        module_header_name = module.replace(".", ": ").replace("_", " ").title().replace("Darts", "DARTS")
        f.write(f"# {module_header_name} Reference\n\n")
        f.writelines(toc)
        f.write(f"::: {module}\n")
        # f.write("\n")

In [ ]:
from collections import defaultdict

navc = defaultdict(dict)
for module in modules:
    if "." in module:
        parent = module.split(".")[0]
        child = module.split(".")[1]
    else:
        parent = module
        child = "index"
    module_docpath = Path("../docs/ref") / module.replace(".", "/")
    module_docfile = module_docpath / "index.md"
    module_header_name = module.replace("_", " ").replace("darts.", "").title().replace("Darts", "DARTS")
    nav = f"\t- {module_header_name}:\n"
    nav += f"\t\t- {module_docfile.relative_to('../docs')}\n"
    for classname, _ in getmembers(importlib.import_module(module), isclass):
        class_docfile = module_docpath / f"{classname}.md"
        nav += f"\t\t- {classname.replace('_', ' ')}: {class_docfile.relative_to('../docs')}\n"
    for funcname, _ in getmembers(importlib.import_module(module), isfunction):
        function_docfile = module_docpath / f"{funcname}.md"
        nav += f"\t\t- {funcname.replace('_', ' ').title()}: {function_docfile.relative_to('../docs')}\n"
    navc[parent][child] = nav
print(navc)

In [ ]:
navref = "- Reference:\n"
for key, childs in navc.items():
    navref += childs["index"]
    for child, nav in childs.items():
        if child != "index":
            navref += "\n".join([f"\t{line}" for line in nav.split("\n") if line])
        navref += "\n"
print(navref)